In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
os.chdir('../src')


In [4]:
import pandas as pd
df = pd.read_parquet("../data/raw/raw_data.parquet")



In [21]:
from utils.data_processor import DataCleaner

cleaner = DataCleaner()



In [45]:
def preprocessing(df):
    cleaner = DataCleaner()
    df = cleaner.filter_null(df, "offer_price")
    df = cleaner.filter_null(df, "original_price")
    df["offer_price"] = df["offer_price"].apply(lambda x: cleaner.clean_price(x))
    return df

df_final = preprocessing(df)
df_final.head()

,description,brand,original_price,offer_price
2,zapatillas originales hombre nike tiempo legen...,NIKE,"S/\n289\n,\n90",173.94
7,zapatillas nike air force 1 07 100% original ...,NIKE,S/\n559,475.15
8,zapatillas nike air force 1 ´07 cw2288-111,NIKE,S/\n660,561.00
14,zapatillas reebok niños royal prime 20 2v 1002...,REEBOK,"S/\n139\n,\n90",94.00
21,zapatillas nike court legacy lift urbanas para...,NIKE,S/\n419,314.25


In [46]:
texto = df_final.reset_index(drop = True).loc[1,"description"]
def clean_text(text):
    """
    Clean text by removing special characters and converting to lowercase.
    
    Args:
        text (str): Text to clean
        
    Returns:
        str: Cleaned text in lowercase
    """
    if not isinstance(text, str) or pd.isna(text):
        return None
        
    # Remove special characters and convert to lowercase
    chars_to_remove = ["|", "\\", "/", ",", "."]
    for char in chars_to_remove:
        text = text.replace(char, "")
    
    return text.lower().strip()



df_final["description"] = df_final["description"].apply(cleaner.clean_text)
df_final

,description,brand,original_price,offer_price
2,zapatillas originales hombre nike tiempo legen...,NIKE,"S/\n289\n,\n90",173.94
7,zapatillas nike air force 1 07 100% original ...,NIKE,S/\n559,475.15
8,zapatillas nike air force 1 ´07 cw2288-111,NIKE,S/\n660,561.00
14,zapatillas reebok niños royal prime 20 2v 1002...,REEBOK,"S/\n139\n,\n90",94.00
21,zapatillas nike court legacy lift urbanas para...,NIKE,S/\n419,314.25
...,...,...,...,...
43,zapatillas adidas urbanas grand court 20 k id...,ADIDAS,S/\n189,127.00
44,zapatillas adidas unisex urbanas breaknet 20 e...,ADIDAS,S/\n179,121.00
45,zapatilla puma caven 20 392291-04 para hombre,PUMA,S/\n345,248.40
48,zapatillas adidas hombre urbanas grand court 2...,ADIDAS,S/\n279,182.00


In [47]:
# Tokenization

from nltk.tokenize import word_tokenize
import spacy
def lemmatize_text(text):
    """
    Lemmatize Spanish text using spaCy.
    
    Args:
        text (str): Text to lemmatize
        
    Returns:
        str: Lemmatized text
    """
    # Cargar el modelo en español de spaCy
    nlp = spacy.load("es_core_news_sm")
    
    # Procesar el texto
    doc = nlp(text.lower())
    
    # Obtener los lemas, ignorando stopwords y puntuación
    lemmas = [
        token.lemma_ for token in doc 
        if not token.is_stop and not token.is_punct
    ]
    
    # Unir los lemas en un string
    return " ".join(lemmas)


df_final["description"] = df_final["description"].apply(word_tokenize)
df_final["description_lematized"] = df_final["description"].apply(lambda x: [lemmatize_text(token) for token in x])
df_final
# Stopwords



,description,brand,original_price,offer_price,description_lematized
2,"[zapatillas, originales, hombre, nike, tiempo,...",NIKE,"S/\n289\n,\n90",173.94,"[zapatilla, original, hombre, nike, tiempo, le..."
7,"[zapatillas, nike, air, force, 1, 07, 100, %, ...",NIKE,S/\n559,475.15,"[zapatilla, nike, air, force, 1, 07, 100, , or..."
8,"[zapatillas, nike, air, force, 1, ´07, cw2288-...",NIKE,S/\n660,561.00,"[zapatilla, nike, air, force, 1, ´ 07, cw2288-..."
14,"[zapatillas, reebok, niños, royal, prime, 20, ...",REEBOK,"S/\n139\n,\n90",94.00,"[zapatilla, reebok, niño, royal, prime, 20, 2v..."
21,"[zapatillas, nike, court, legacy, lift, urbana...",NIKE,S/\n419,314.25,"[zapatilla, nike, court, legacy, lift, urbano,..."
...,...,...,...,...,...
43,"[zapatillas, adidas, urbanas, grand, court, 20...",ADIDAS,S/\n189,127.00,"[zapatilla, adida, urbano, grand, court, 20, k..."
44,"[zapatillas, adidas, unisex, urbanas, breaknet...",ADIDAS,S/\n179,121.00,"[zapatilla, adida, unisex, urbano, breaknet, 2..."
45,"[zapatilla, puma, caven, 20, 392291-04, para, ...",PUMA,S/\n345,248.40,"[zapatilla, puma, cavir, 20, 392291-04, , hombre]"
48,"[zapatillas, adidas, hombre, urbanas, grand, c...",ADIDAS,S/\n279,182.00,"[zapatilla, adida, hombre, urbano, grand, cour..."


In [ ]:
import nltk
def remove_stopwords(text):
    """
    Remove Spanish stopwords from text.
    
    Args:
        text (str): Text to process (should be a string with words separated by spaces)
        
    Returns:
        str: Text with stopwords removed
    """
    # Descargar stopwords si no están descargadas


    stopwords = nltk.download('stopwords')
    
    # Obtener stopwords en español
    stop_words = nltk.corpus.stopwords.words('spanish')
     
    # Filtrar stopwords y unir las palabras
    filtered_words = [word for word in text if word not in stop_words]
    return " ".join(filtered_words)

df_final["description_stopwords_removed"] = df_final["description_lematized"].apply(remove_stopwords)
df_final


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aladelca/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aladelca/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aladelca/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aladelca/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aladelca/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aladelca/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aladelca/nltk_data...
[nltk_data]   Package stopwords is alrea

,description,brand,original_price,offer_price,description_lematized,description_stopwords_removed
2,"[zapatillas, originales, hombre, nike, tiempo,...",NIKE,"S/\n289\n,\n90",173.94,"[zapatilla, original, hombre, nike, tiempo, le...",zapatilla original hombre nike tiempo legend 1...
7,"[zapatillas, nike, air, force, 1, 07, 100, %, ...",NIKE,S/\n559,475.15,"[zapatilla, nike, air, force, 1, 07, 100, , or...",zapatilla nike air force 1 07 100 original dd...
8,"[zapatillas, nike, air, force, 1, ´07, cw2288-...",NIKE,S/\n660,561.00,"[zapatilla, nike, air, force, 1, ´ 07, cw2288-...",zapatilla nike air force 1 ´ 07 cw2288-111
14,"[zapatillas, reebok, niños, royal, prime, 20, ...",REEBOK,"S/\n139\n,\n90",94.00,"[zapatilla, reebok, niño, royal, prime, 20, 2v...",zapatilla reebok niño royal prime 20 2v 100202...
21,"[zapatillas, nike, court, legacy, lift, urbana...",NIKE,S/\n419,314.25,"[zapatilla, nike, court, legacy, lift, urbano,...",zapatilla nike court legacy lift urbano mujer
...,...,...,...,...,...,...
43,"[zapatillas, adidas, urbanas, grand, court, 20...",ADIDAS,S/\n189,127.00,"[zapatilla, adida, urbano, grand, court, 20, k...",zapatilla adida urbano grand court 20 k id7870
44,"[zapatillas, adidas, unisex, urbanas, breaknet...",ADIDAS,S/\n179,121.00,"[zapatilla, adida, unisex, urbano, breaknet, 2...",zapatilla adida unisex urbano breaknet 20 ie3790
45,"[zapatilla, puma, caven, 20, 392291-04, para, ...",PUMA,S/\n345,248.40,"[zapatilla, puma, cavir, 20, 392291-04, , hombre]",zapatilla puma cavir 20 392291-04 hombre
48,"[zapatillas, adidas, hombre, urbanas, grand, c...",ADIDAS,S/\n279,182.00,"[zapatilla, adida, hombre, urbano, grand, cour...",zapatilla adida hombre urbano grand court 20 i...


In [58]:
stopwords = nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('spanish')
stop_words

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aladelca/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'no',
 'una',
 'su',
 'al',
 'lo',
 'como',
 'más',
 'pero',
 'sus',
 'le',
 'ya',
 'o',
 'este',
 'sí',
 'porque',
 'esta',
 'entre',
 'cuando',
 'muy',
 'sin',
 'sobre',
 'también',
 'me',
 'hasta',
 'hay',
 'donde',
 'quien',
 'desde',
 'todo',
 'nos',
 'durante',
 'todos',
 'uno',
 'les',
 'ni',
 'contra',
 'otros',
 'ese',
 'eso',
 'ante',
 'ellos',
 'e',
 'esto',
 'mí',
 'antes',
 'algunos',
 'qué',
 'unos',
 'yo',
 'otro',
 'otras',
 'otra',
 'él',
 'tanto',
 'esa',
 'estos',
 'mucho',
 'quienes',
 'nada',
 'muchos',
 'cual',
 'poco',
 'ella',
 'estar',
 'estas',
 'algunas',
 'algo',
 'nosotros',
 'mi',
 'mis',
 'tú',
 'te',
 'ti',
 'tu',
 'tus',
 'ellas',
 'nosotras',
 'vosotros',
 'vosotras',
 'os',
 'mío',
 'mía',
 'míos',
 'mías',
 'tuyo',
 'tuya',
 'tuyos',
 'tuyas',
 'suyo',
 'suya',
 'suyos',
 'suyas',
 'nuestro',
 'nuestra',
 'nuestros',
 'nuestras',
 'vuestro'

In [61]:
from pipeline import pipeline

df = pipeline(df)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aladelca/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aladelca/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aladelca/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aladelca/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aladelca/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aladelca/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aladelca/nltk_data...
[nltk_data]   Package stopwords is alrea

In [62]:
df

,description,brand,original_price,offer_price,description_lematized,text_cleaned
2,"[zapatillas, originales, hombre, nike, tiempo,...",NIKE,"S/\n289\n,\n90",173.94,"[zapatilla, original, hombre, nike, tiempo, le...",zapatillas originales hombre nike tiempo legen...
7,"[zapatillas, nike, air, force, 1, 07, 100, %, ...",NIKE,S/\n559,475.15,"[zapatilla, nike, air, force, 1, 07, 100, , or...",zapatillas nike air force 1 07 100 % original ...
8,"[zapatillas, nike, air, force, 1, ´07, cw2288-...",NIKE,S/\n660,561.00,"[zapatilla, nike, air, force, 1, ´ 07, cw2288-...",zapatillas nike air force 1 ´07 cw2288-111
14,"[zapatillas, reebok, niños, royal, prime, 20, ...",REEBOK,"S/\n139\n,\n90",94.00,"[zapatilla, reebok, niño, royal, prime, 20, 2v...",zapatillas reebok niños royal prime 20 2v 1002...
21,"[zapatillas, nike, court, legacy, lift, urbana...",NIKE,S/\n419,314.25,"[zapatilla, nike, court, legacy, lift, urbano,...",zapatillas nike court legacy lift urbanas mujer
...,...,...,...,...,...,...
43,"[zapatillas, adidas, urbanas, grand, court, 20...",ADIDAS,S/\n189,127.00,"[zapatilla, adida, urbano, grand, court, 20, k...",zapatillas adidas urbanas grand court 20 k id7870
44,"[zapatillas, adidas, unisex, urbanas, breaknet...",ADIDAS,S/\n179,121.00,"[zapatilla, adida, unisex, urbano, breaknet, 2...",zapatillas adidas unisex urbanas breaknet 20 i...
45,"[zapatilla, puma, caven, 20, 392291-04, para, ...",PUMA,S/\n345,248.40,"[zapatilla, puma, cavir, 20, 392291-04, , hombre]",zapatilla puma caven 20 392291-04 hombre
48,"[zapatillas, adidas, hombre, urbanas, grand, c...",ADIDAS,S/\n279,182.00,"[zapatilla, adida, hombre, urbano, grand, cour...",zapatillas adidas hombre urbanas grand court 2...
